In [1]:
!pip3 install matplotlib
!pip3 install tensorflow
!pip3 install scikit-learn

In [2]:
from keras.datasets import fashion_mnist

In [3]:
import numpy as np

In [ ]:
(x_train_full, y_train_full),( x_test_full, y_test_full) = fashion_mnist.load_data()
print(y_test_full.shape) #no need to flatten
print(y_train_full.shape) 

#y_train_full = y_train_full.flatten()
#ytest_full =  y_test_full.flatten()
x_full= np.concatenate([x_test_full, x_train_full])

y_full= np.concatenate([y_test_full,y_train_full])


(10000,)
(60000,)
